In [ ]:
import os

import numpy as np
import scipy.sparse as sps
from pyvis.network import Network
from particle import Particle
import vector

from cluster_gnn.data import internal as Data
from cluster_gnn.features import build_features as Features

In [ ]:
ROOT_DIR = os.path.expanduser('~/projects/cluster_gnn/')

with Data.EventLoader(ROOT_DIR + 'data/processed/events.hdf5', 'wboson') as evts:
    # set data object state to point at first event
    num_evts = len(evts)
    evt_num = np.random.randint(num_evts)
    evts.set_evt(evt_num)
    # form weighted edges
    pmu = evts.get_pmu()
    signal = evts.get_signal()
    sps_adj = sps.coo_matrix(
        Features.knn_adj(
            Features.deltaR_aff(pmu),
            k=8,
            weighted=True,
            dtype=np.float32
            )
        )
    edge_idxs = zip(tuple(int(idx) for idx in sps_adj.row),
                    tuple(int(idx) for idx in sps_adj.col),
                    tuple(1.0 / (float(wt) + 0.1) for wt in sps_adj.data)
                    )
    # get data from particle ids
    pcls = [Particle.from_pdgid(pdg) for pdg in evts.get_pdg()]
    labels = list(map(lambda pcl: pcl.name, pcls))
    ids = list(range(len(pcls)))
    node_vals = Features._array_to_vec(pmu).pt
    node_vals = node_vals * 10.0 / node_vals.max()
    node_vals = tuple(float(val) for val in node_vals)
    # identify which are W boson
    groups = list(signal.astype(np.uint8))
    print(evts.get_evt_name())
net = Network(height=600, width=800, notebook=True, directed=False)
net.toggle_hide_edges_on_drag(False)
net.barnes_hut()
net.add_nodes(ids,
          label=labels,
          value=node_vals,
          color=['#162347' if group == 0 else '#dd4b39' for group in groups],
          )
net.add_edges(edge_idxs)
net.show('ex.html')

Could charge be incorperated, and then summed at the end for comparison against the charge of the initial particle?
_eg._ the W0 boson has Q = 0, so if the sum of the charges of the predicted particles is very different from zero, the model could be penalised?

In [ ]:
tot_charge = 0.0
for i in np.argwhere(signal).reshape(-1):
    pcl = pcls[i]
    tot_charge += pcl.charge
print(f'The total charge of final state is {tot_charge}')

In [46]:
import numpy as np

from cluster_gnn import ROOT_DIR
from cluster_gnn.data import internal as Data
from cluster_gnn.data import pdg

lookup_pdg = pdg.LookupPDG()

with Data.EventLoader(ROOT_DIR + '/data/processed/events.hdf5', 'wboson') as evts:
    evt_num = np.random.randint(len(evts))
    evts.set_evt(evt_num)
    pmu = evts.get_pmu()
    pdgs = evts.get_pdg()
    signal = evts.get_signal()
    seed_pdg = evts.get_ue_pcls('signal_pcl')

charges = lookup_pdg.get_charge(pdgs)
cum_charge = np.sum(np.array(charges)[signal])
seed_charge = lookup_pdg.get_charge([seed_pdg])

print(f'seed charge is {seed_charge}, and cumulative constit charge is {cum_charge}')

seed charge is [1.0], and cumulative constit charge is -1.0


In [61]:
import torch
q_t = torch.tensor(charges)
q_t = q_t.reshape(-1, 1)
pmu_t = torch.tensor(pmu)

In [63]:
print(q_t.shape)
print(pmu_t.shape)

torch.Size([226, 1])
torch.Size([226, 4])


In [87]:
feat = torch.cat([q_t, pmu_t], dim=1)

In [88]:
fancy_idx = torch.from_numpy(np.argwhere(signal).reshape(-1))

In [90]:
feat[fancy_idx, 0].sum()

tensor(-1.)

In [92]:
edge_attrs = torch.rand(20)

In [93]:
mask = edge_attrs.ge(0.5)

tensor([False,  True, False, False,  True, False,  True,  True, False, False,
        False,  True,  True,  True, False,  True,  True, False, False, False])